# Engine

> Adapted from SCM/lib/engine.py

In [ ]:
#| default_exp core.engine

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import os
import shutil
import torch
import numpy as np
from loguru import logger
from sklearn.metrics import average_precision_score

from scm_nbdev.datasets.cub import CUBDataset
from scm_nbdev.datasets.imagenet import ImageNetDataset

In [ ]:
#| export
def create_data_loader(cfg, root_dir):
    """ Create data_loaders for training and validation
    :param cfg: hyperparameter configuration
    :param root_dir: dataset root path
    :return:
    """
    logger.info('preparing data...')
    if cfg.DATA.DATASET == 'CUB':
        train_loader = torch.utils.data.DataLoader(
            CUBDataset(root=root_dir, cfg=cfg, is_train=True),
            batch_size=cfg.TRAIN.BATCH_SIZE, shuffle=True, num_workers=cfg.BASIC.NUM_WORKERS, pin_memory=True)
        test_loader = torch.utils.data.DataLoader(
            CUBDataset(root=root_dir, cfg=cfg, is_train=False),
            batch_size=cfg.TEST.BATCH_SIZE, shuffle=False, num_workers=cfg.BASIC.NUM_WORKERS, pin_memory=True)
        # val_loader = torch.utils.data.DataLoader(
        #     CUBDataset(root=root_dir, cfg=cfg, is_train=False, val=True),
        #     batch_size=cfg.TEST.BATCH_SIZE, shuffle=False, num_workers=cfg.BASIC.NUM_WORKERS, pin_memory=True)
        val_loader = None
    elif cfg.DATA.DATASET == 'ImageNet':
        
        train_loader = torch.utils.data.DataLoader(
            ImageNetDataset(root=root_dir, cfg=cfg, is_train=True),
            batch_size=cfg.TRAIN.BATCH_SIZE, shuffle=True, num_workers=cfg.BASIC.NUM_WORKERS, pin_memory=True)
        test_loader = torch.utils.data.DataLoader(
            ImageNetDataset(root=root_dir, cfg=cfg, is_train=False),
            batch_size=cfg.TEST.BATCH_SIZE, shuffle=False, num_workers=cfg.BASIC.NUM_WORKERS, pin_memory=True)
        val_loader = torch.utils.data.DataLoader(
            ImageNetDataset(root=root_dir, cfg=cfg, is_train=False, val=True),
            batch_size=cfg.TEST.BATCH_SIZE, shuffle=False, num_workers=cfg.BASIC.NUM_WORKERS, pin_memory=True)
                
    logger.info('done loading..')
    return train_loader, test_loader, val_loader

In [ ]:
#| export
def str_gpus(ids):
    str_ids = ''
    for i in ids:
        str_ids =  str_ids + str(i)
        str_ids =  str_ids + ','
    return str_ids

def map_sklearn(labels, results):
    _map = average_precision_score(labels, results, average="micro")
    return _map


#must be moved to external lr_scheduler
def adjust_learning_rate(optimizer, epoch, cfg):
    """"Sets the learning rate to the initial LR decayed by lr_factor"""
    lr_decay = cfg.SOLVER.LR_FACTOR**(sum(epoch > np.array(cfg.SOLVER.LR_STEPS)))
    lr = cfg.SOLVER.START_LR * lr_decay
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr * param_group['lr_mult']


def adjust_lr_by_scheduler(lr_scheduler, optimizer, curr_iter):
    """ Adjust the learning rate of model parameters by scheduler
    :param optimizer: optimizer (e.g. SGD, AdamW, Adam)
    :param epoch: training epoch
    :param curr_iter: current iteration
    :return:
    """
    lr = lr_scheduler.update_lr(curr_iter)
    for param_group in optimizer.param_groups:
        param_group["lr"] = lr

def save_checkpoint(state, save_dir, epoch, is_best):
    filename = os.path.join(save_dir, 'ckpt_'+str(epoch)+'.pth.tar')
    torch.save(state, filename)
    if is_best:
        best_name = os.path.join(save_dir, 'model_best.pth.tar')
        shutil.copyfile(filename, best_name)


class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count


#maybe moved to external metrics module
def accuracy(output, target, topk=(1,)):
    """ Computes the precision@k for the specified values of k
    :param output: tensor of shape B x K, predicted logits of image from model
    :param target: tensor of shape B X 1, ground-truth logits of image
    :param topk: top predictions
    :return: list of precision@k
    """
    maxk = max(topk)
    batch_size = target.size(0)

    _, pred = output.topk(1, 1, True, True)
    pred = pred.t()
    correct = pred.eq(target.view(1, -1).expand_as(pred))

    res = []
    for k in topk:
        correct_k = correct[:k].reshape(-1).float().sum(0)
        res.append(correct_k.mul_(100.0 / batch_size))
    return res


def list2acc(results_list):
    """
    :param results_list: list contains 0 and 1
    :return: accuarcy
    """
    accuarcy = results_list.count(1)/len(results_list)
    return accuarcy

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()